In [ ]:
#!pip install -q openai==1.59.8 google-generativeai==0.8.3 llama-index==0.12.11 llama-index-llms-gemini==0.4.3

In [1]:
# Read data

import csv

rows = []

# Load the CSV file
with open("./2_Using_LlamaIndex/mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
  csv_reader = csv.reader(file)

  for idx, row in enumerate(csv_reader):
    if idx == 0: continue; # Skip header row
    rows.append(row)

# The number of artickes in the dataset.
print("number of articles:", len(rows))

number of articles: 14


In [2]:
# Load the LlamaIndex framework and create Document objects.
from llama_index.core import Document

# Convert the texts to Document objects so the LlamaIndex framework can process them.
documents = [Document(text=row[1]) for row in rows]

In [3]:
from dotenv import load_dotenv
import os
import openai
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
# Set the API key for the OpenAI client
openai.api_key = api_key

g_api_key = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=g_api_key)

In [4]:
# Build index / generate embeddings using OpenAI embedding model
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding

# Build index / generate embeddings using OpenAI embedding model
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=OpenAIEmbedding(model="text-embedding-3-small"),
    transformations=[SentenceSplitter(chunk_size=768, chunk_overlap=64)],
    show_progress=True,
)

Parsing nodes:   0%|          | 0/14 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/14 [00:00<?, ?it/s]

In [7]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("How many parameters does the LLaMA2 model have?")
print(response)

The LLaMA2 model has parameters ranging from 7 billion to 70 billion across its suite of models.


In [10]:
# # Define a query engine that is responsible for retrieving related pieces of text,
# # and using a LLM to formulate the final answer.
# from llama_index.llms.gemini import Gemini
# from llama_index.core import Settings
# Settings.llm = Gemini(api_key=g_api_key)

# llm = Gemini(model="models/gemini-1.5-flash", temperature=1, max_tokens=512, api_key=g_api_key)

# query_engine = index.as_query_engine(llm=llm)

In [11]:
# # # Generate a response to a query.
# # # Example function to handle the query and extract the response
# # def handle_query(query_engine, query):
# #     try:
# #         # Perform the query
# #         response = query_engine.query(query)
        
# #         # Navigate through the nested structure based on the new SDK response format
# #         if response and 'candidates' in response and len(response['candidates']) > 0:
# #             candidate = response['candidates'][0]
# #             if 'content' in candidate and 'parts' in candidate['content'] and len(candidate['content']['parts']) > 0:
# #                 text = candidate['content']['parts'][0]['text']
# #                 return text
# #             else:
# #                 raise ValueError("Response format is incorrect or missing expected content")
# #         else:
# #             raise ValueError("No candidates found in response")
    
# #     except Exception as e:
# #         print(f"An error occurred: {e}")
# #         return None

# # # Example usage
# # query_text = "How many parameters does the LLaMA2 model have?"
# # result_text = handle_query(query_engine, query_text)
# # print("Query Result:", result_text)

# response = query_engine.query("How many parameters LLaMA2 model has?")
# print(response)